# Crop simulation

Doing imports 

In [46]:
%matplotlib inline
#%matplotlib notebook
import sys
import pcse
import pandas as pd 
import numpy as np
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
print("This notebook was built with:")
print(f"python version: {sys.version}")
print(f"PCSE version: {pcse.__version__}")

This notebook was built with:
python version: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
PCSE version: 6.0.9


## Transform the data into the right units


| Identifier | Parameter   | Description                              | From Unit  | To Unit           | Dataset |
|------------|-------------|------------------------------------------|------------|-------------------|---------|
| TXK        | TMAX        | Daily maximum temperature                | °C         | °C                | KL      |
| TNK        | TMIN        | Daily minimum temperature                | °C         | °C                | KL      |
| VPM        | VAP         | Mean daily vapour pressure               | hPa        | hPa               | KL      |
| FM         | WIND        | Mean daily wind speed at 2 m above ground level | m/s        | m/s       | KL      |
| RSK        | RAIN        | Precipitation                            | mm         | cm   | KL      |
| FG_STRAHL  | IRRAD       | Daily global radiation                   | J/cm^2     | J/m^{-2} | Solar   |
| SHK_TAG    | SNOWDEPTH   | Depth of snow cover (optional)           | cm         | cm                | KL      |

#### Climate data

In [59]:
# Read the text file into a DataFrame
data_bremen_kl = pd.read_csv('data_kl/bremen_daily_kl/tageswerte_KL_00691_18900101_20231231_hist/produkt_klima_tag_18900101_20231231_00691.txt', delimiter=';', skipfooter=1, engine='python')

data_chemnitz_kl = pd.read_csv('data_kl/chemnitz_daily_kl/tageswerte_KL_00853_18820101_20231231_hist/produkt_klima_tag_18820101_20231231_00853.txt', delimiter=';', skipfooter=1, engine='python')

# Convert units -> mm to cm, ensuring column names are stripped of extra spaces
data_bremen_kl.columns = data_bremen_kl.columns.str.strip()
data_chemnitz_kl.columns = data_chemnitz_kl.columns.str.strip()

# Replace -999 with NaN
data_bremen_kl.replace(-999, np.nan, inplace=True)
data_chemnitz_kl.replace(-999, np.nan, inplace=True)


#### Temperature data

In [60]:
# Read the text file into a DataFrame
data_bremen_solar = pd.read_csv('data_solar/bremen/tageswerte_ST_00691_row/produkt_st_tag_20090101_20250331_00691.txt', delimiter=';', skipfooter=1, engine='python')

data_chemnitz_solar = pd.read_csv('data_solar/chemnitz/tageswerte_ST_00853_row/produkt_st_tag_19810101_20250331_00853.txt', delimiter=';', skipfooter=1, engine='python')

# Replace -999 with NaN
data_bremen_solar.replace(-999, np.nan, inplace=True)
data_chemnitz_solar.replace(-999, np.nan, inplace=True)

In [ ]:
# find the rows without NaNs

# Create a boolean mask for rows without NaN values
mask_bremen = data_bremen_solar.notna().all(axis=1)
mask_chemnitz = data_chemnitz_solar.notna().all(axis=1)

# Filter and print the rows without NaN values
data_bremen_solar_cleaned = data_bremen_solar[mask_bremen]
data_chemnitz_solar_cleaned = data_chemnitz_solar[mask_chemnitz]

print("Bremen Solar Data without NaNs:")
print(data_bremen_solar_cleaned)

print("Chemnitz Solar Data without NaNs:")
print(data_chemnitz_solar_cleaned)

### Transform to PCSE data format

In [61]:
# Map and transform the data to PCSE format
data_bremen_pcse = pd.DataFrame({
    #'DAY': pd.to_datetime(data_bremen_kl['MESS_DATUM'], format='%Y%m%d'),
    'DAY': data_bremen_kl['MESS_DATUM'],
    'TMIN': data_bremen_kl['TNK'],  # Example: map to minimum temperature
    'TMAX': data_bremen_kl['TXK'],  # Example: map to maximum temperature
    'VAP': data_bremen_kl['VPM'],  # Example: map to vapor pressure
    'RAIN': data_bremen_kl['RSK'] / 10,  # Convert mm to cm if needed
    'IRRAD': data_bremen_solar['FG_STRAHL'] * 10000,  # Convert J/cm² to J/m²
    'WIND': data_bremen_kl['FM'],  # Assuming this is the wind speed
})

In [62]:
# Map and transform the data to PCSE format
data_chemnitz_pcse = pd.DataFrame({
    #'DAY': pd.to_datetime(data_chemnitz_kl['MESS_DATUM'], format='%Y%m%d'),
    'DAY': data_chemnitz_kl['MESS_DATUM'],
    'TMIN': data_chemnitz_kl['TNK'],  # Example: map to minimum temperature
    'TMAX': data_chemnitz_kl['TXK'],  # Example: map to maximum temperature
    'VAP': data_chemnitz_kl['VPM'],  # Example: map to vapor pressure
    'RAIN': data_chemnitz_kl['RSK'] / 10,  # Convert mm to cm if needed
    'IRRAD': data_chemnitz_solar['FG_STRAHL'] * 10000,  # Convert J/cm² to J/m²
    'WIND': data_chemnitz_kl['FM'],  # Assuming this is the wind speed
})

#print(data_chemnitz_pcse)

# Remove rows with any NaN values
data_cleaned = data_chemnitz_pcse.dropna()
print(data_cleaned)

# Filter the data for a specific date range (e.g., from 2009-01-01 to 2023-12-31)
#data_chemnitz_filtered = data_chemnitz_pcse[data_chemnitz_pcse['DAY'] >= 20090101]
#data_chemnitz_filtered.to_csv('data_chemnitz_filtered.csv', index=False, na_rep='NaN')


# Write to CSV file without site characteristics
#data_chemnitz_pcse.to_csv('data_chemnitz_pcse.csv', index=False, na_rep='NaN')


Empty DataFrame
Columns: [DAY, TMIN, TMAX, VAP, RAIN, IRRAD, WIND]
Index: []


In [ ]:
# Site characteristics as a string
site_characteristics = """## Site Characteristics
Country     = 'Germany'
Station     = 'Chemnitz'
Description = 'Observed data from Station Chemnitz'
Source      = 'Your Data Source'
Contact     = 'Your Contact'
Longitude = 12.92; Latitude = 50.83; Elevation = 300; AngstromA = 0.25; AngstromB = 0.50; HasSunshine = False
## Daily weather observations (missing values are NaN)
"""

# Convert DataFrame to CSV string
csv_data = data_chemnitz_pcse.to_csv(index=False, na_rep='NaN')

# Combine site characteristics and CSV data
full_csv_content = site_characteristics + csv_data

# Write to CSV file
with open('chemnitz_withMeta.csv', 'w') as f:
    f.write(full_csv_content)

print("CSV file with site characteristics created.")

In [55]:
from pcse.fileinput import CSVWeatherDataProvider

# Load the weather data provider
weatherfile = "chemnitz_withMeta.csv"
weatherdataprovider = CSVWeatherDataProvider(weatherfile, delimiter=',', dateformat='%Y%m%d', ETmodel='PM', force_reload=False)
print(weatherdataprovider)

AssertionError: Invalid sunshine duration value (nan) encountered at day 1882-01-01

In [ ]:
import pandas as pd

# Sample metadata for Bremen
metadata = {
    'Country': 'Germany',
    'Station': 'Bremen',
    'Description': 'Observed data from Station Bremen',
    'Source': 'Your Data Source',
    'Contact': 'Your Contact',
    'Longitude': 8.7981,
    'Latitude': 53.0451,
    'Elevation': 4.04,
    'AngstromA': 0.25,
    'AngstromB': 0.50,
    'HasSunshine': False
}

# Convert IRRAD to kJ/m² (from J/m²)
#data_bremen_pcse['IRRAD'] = data_bremen_pcse['IRRAD'] / 1000

# Convert VAP from hPa to kPa
#data_bremen_pcse['VAP'] = data_bremen_pcse['VAP'] / 10

# Prepare CSV content
csv_content = []

# Add metadata
csv_content.append("## Site Characteristics")
for key, value in metadata.items():
    csv_content.append(f"{key} = '{value}'")

# Add data header
csv_content.append("## Daily weather observations (missing values are NaN)")
csv_content.append(",".join(data_bremen_pcse.columns))

# Add data rows
for index, row in data_bremen_pcse.iterrows():
    csv_content.append(",".join(map(str, row.values)))

# Write to CSV
with open('bremen_weather_data.csv', 'w') as f:
    for line in csv_content:
        f.write(line + "\n")

print("CSV file created.")

CSV file created.


### Test weather data provider with example exel file

In [19]:
from pcse.fileinput import ExcelWeatherDataProvider
# Ensure the file is properly created as an Excel file
# data_chemnitz_pcse.to_excel("chemnitz_daily.xlsx", index=False)

# Load the weather data provider
weatherfile = "nl1.xlsx"
weatherdataprovider = ExcelWeatherDataProvider(weatherfile)
print(weatherdataprovider)

Failed reading row: 13. Skipping...
Weather data provided by: ExcelWeatherDataProvider
--------Description---------
Weather data for:
Country: Netherlands
Station: Wageningen, Location Haarweg
Description: Observed data from Station Haarweg in Wageningen
Source: Meteorology and Air Quality Group, Wageningen University
Contact: Peter Uithol
----Site characteristics----
Elevation:    7.0
Latitude:  51.970
Longitude:  5.670
Data available for 2004-01-02 - 2008-12-31
Number of missing days: 32



# Soil and Agro data

In [50]:
#crop, soil, site and agro
from pcse.util import DummySoilDataProvider
from pcse.input import WOFOST72SiteDataProvider 
import yaml
from pcse.base import ParameterProvider

#load crop file
with open('data/crop/potato.yaml', 'r') as file: #alternative: from pcse.input import YAMLCropDataProvider
    cropd = yaml.safe_load(file)

#load dummy soil file
soild = DummySoilDataProvider() #use dummy soil: https://pcse.readthedocs.io/en/stable/code.html#pcse.util.DummySoilDataProvider

#load some standard site
sited = WOFOST72SiteDataProvider(WAV=10) #https://pcse.readthedocs.io/en/stable/code.html#wofost72sitedataprovider

#put everything together with ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

#agro file (is not completed yet)
with open('data/agro/potato_agro_test.yaml', 'r') as file:
    agromanagement = yaml.safe_load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'data/crop/potato.yaml'